# 3 External clustering evaluation

## 3.1 Mathmatical evaluation: SCI

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets

x = np.loadtxt("./data/evalua_data/graph_data_BvsD.txt") # v1: expression matrix/feature matrix, data matrix that we perform clustering on 
y_BvsD_subcel_Kmeans = np.loadtxt('./data/evalua_data/Kmeans_10cluster_BvsD_subcel_types.txt')
y_BvsD_Kmeans = np.loadtxt('./data/evalua_data/Kmeans_10cluster_BvsD_types.txt')
y_BvsD_dgi = np.loadtxt('./data/evalua_data/dgi_10cluster_BvsD_types.txt')
y_BvsD_subcel_dgi = np.loadtxt('./data/evalua_data/dgi_subcel_10cluster_BvsD_types.txt')
y_BvsD_HC = np.loadtxt('./data/evalua_data/HC_10cluster_BvsD_types.txt')
# y_BvsD_HC = np.loadtxt('../output_data/types_BvsD_HC.txt')

from sklearn.metrics import silhouette_score
print("Kmeans without subcellular information:", metrics.silhouette_score(x, y_BvsD_Kmeans, metric='euclidean'))
print("Kmeans with subcellular information:", metrics.silhouette_score(x, y_BvsD_subcel_Kmeans, metric='euclidean')) 
print("DGI without subcellular information:", metrics.silhouette_score(x, y_BvsD_dgi, metric='euclidean'))
print("DGI with subcellular information:", metrics.silhouette_score(x, y_BvsD_subcel_dgi, metric='euclidean'))
print("HC:", metrics.silhouette_score(x, y_BvsD_HC, metric='euclidean'))


# print("Kmeans:", silhouette_score(x, y4_BvsD_Kmeans)) # Kmeans
# print("HC:", silhouette_score(x, y5_BvsD_HC)) # HC



Kmeans without subcellular information: 0.1831181694401295
Kmeans with subcellular information: 0.13457301691358345
DGI without subcellular information: 0.18504816673000304
DGI with subcellular information: 0.19256539442997084
HC: 0.0018333779252142142


## 3.2 Biological evaluation: (Function assignment score) FAS

In [ ]:
# import modules
import pandas as pd 
import numpy as np
import os
# Load data
cluster_label = pd.read_csv('./data/label_processed/tsne_BvsD_labels_all_7cluster.csv')
fpkm = pd.read_table('./data/raw_dataset/gene_fpkm_all_samples.txt')
annotation = pd.read_csv('./data/annotations/Annotation_Expre.csv')
kegg_anno = pd.read_csv('./data/annotations/kegg_annotation_all.csv')

In [ ]:
# define FAS function
def FAS(cluster_label, fpkm, annotation, kegg_anno):
    # match data matrix
    matched = pd.merge(cluster_label, annotation, left_on = 'gene_id', right_on = 'gene_id', how = 'outer', indicator = False)
    matched_kegg = pd.merge(matched, kegg_anno, on = 'gene_id', how = 'inner', indicator = False)
    
    # grep functions
    photosynthesis = matched_kegg[matched_kegg['level3_pathway_name'].str.contains("Photosynthesis")]
    peroxisome = matched_kegg[matched_kegg['level3_pathway_name'].str.contains("Peroxisome")]
    fun = peroxisome
    
    # create data frame from `fun$DGI_label` column
    data = pd.DataFrame({'Var1': fun['DGI_label']})
    data = data.groupby('Var1').size().reset_index(name='Freq')

    # create data frame from `matched_kegg$DGI_label` column
    ref = pd.DataFrame({'Var1': matched_kegg['DGI_label']})
    ref = ref.groupby('Var1').size().reset_index(name='Freq')

    # merge data frames on `Var1` column
    data_ref = pd.merge(data, ref, on='Var1')

    # obtain genes ratio
    result = pd.DataFrame(data_ref['Freq_x'] / data_ref['Freq_y'], columns=['Freq'])
    result_ordered = result.sort_values('Freq', ascending=False)
    
    # calculate FAS (score)
    result_ordered = result_ordered.squeeze()
    if len(result_ordered) > 2:
        score_raw = sum(result_ordered[:2]) - sum(result_ordered[2:])
    else:
        score_raw = sum(result_ordered)
    score = score_raw / (data_ref['Freq_x'].sum() / data_ref['Freq_y'].sum())
    
    return score

In [ ]:
# Utilization
FAS = FAS(cluster_label, fpkm, annotation, kegg_anno)